#### Main Idea: How do we update a certain weight in the network? 
![backprop](backprop1.png)
- Backprop is just a generalization/application of the chain rule from basic calculus
- In the forward pass, we calculate the output by repeatedly applying functions to our inputs, and passing those results through the rest of the layers in our network
- In backprop, we have an incoming/upstream gradient, and apply a local gradient to get the abckpropped derivative

#### Examples of Backprop
- Basic intuition is to break up the large function and interpret it as a composition of several functions. 
- Multiply the input (upstream/incoming derivative) with the function's derivative (i.e. the local gradient) to get the backpropagated gradient

![back2](back2.png)

![back3](back3.png)

- Gate interpretation: add gates pass the gradient through (distribute it), mult gates act as a gradient switcher. 
- What happens when 2 gradient paths converge? The gradients are summed:

![back4](back4.png)
- Here, we have $\frac{dL}{dx} = \frac{dL}{dq_1} + \frac{dL}{dq_2}$. 

#### Derivaitive of scalar with respect to a vector
- When $y$ is a scalar and $x \in R^m$ is a vector, $\nabla_x y \in R^m$ and each element $i$ of the derivative is given by $\frac{dy}{dx_i}$. The gradient is teh same size as teh vector, and each element tells us how much a small change in that direction would change $y$ (i.e. it would change it by $(\nabla_x y)_i \Delta x_i$.

#### Derivative of scalar with respect to a matrix
-  Where $A \in R^{m * n}$, we have $$\nabla_A y = \begin{bmatrix} \frac{dy}{da_1a_1} & ... & \frac{dy}{da_1a_n} \\ ... & ... & ... \\ \frac{dy}{da_ma_1} & ... & \frac{dy}{da_ma_n} \end{bmatrix}$$
- Note that the derivative is of the same shape a A, and $(\nabla_A y)_{i,j}$ tells us how mucha  small change in $A_{i,j}$ will affect the scalar $y$. 
- This notation (where the shape is the same as the value we are differentiating with respect to is known as **denominator layout notation**. 

#### Derivative of vector with respect to a vector
- $\nabla_x y$ where $x \in R^m, y \in R^n$ will be an $m * n$ matrix $J$. Here, $$J = \begin{bmatrix} \frac{dy_1}{dx_1} & ... & \frac{dy_n}{dx_1} \\ ... & ... & ... \\ \frac{dy_1}{dx_m} & ... & \frac{dy_n}{dax_m} \end{bmatrix}$$. 
- Note that since we are in denominator layout notation, the shape is determined by letting the first dim be the dim of what we are differentiating with respct to, and then tacking on the shape of the second element as the next dimensionality. THe vector $x$ that we are differentiating with respect to changes along the columns (so $J \in R^{m * n}$). 
- Example: $\nabla_x Wx$ where $W \in R^{h*n}$ and $x \in R^n$ is given by $W^T$ which is of shape $n * h$. Concretely, here $Wx$ is a vector of shape $h$ and $x$ has shape $n$, so we'd expect an $n * h$ resulting derivative
- Oftentimes you can use this intuition behind the shapes we expect in order to compute the derivative without explicitly writing down teh tensor calculus. 

#### Derivative of a vector with respect to a matrix, or derivative of a matrix with respect to a vector
- These derivatives generally result in 3 dimensional tensors

#### Chain Rule for Vector-Value Functions
- With vector valued functions the chain rule runs right to left in denominator layout notation. 
- Example: We have $x\in R^m, y \in R^n, z \in R^p$. And $y = f(x), z = g(y)$. We want to compute $\frac{dz}{dx}$. From the scalar chain rule, we might write down $\frac{dz}{dx} = \frac{dz}{dy}\frac{dy}{dx}$. 
- But we can see that $\frac{dz}{dy} \in R^{n * p}$ and $\frac{dy}{dx} \in R^{m * n}$, so these shapes don't line up to do the matrix multiplication to compute the derivative.
- The chain rule actually goes right to left, so we have $\frac{dz}{dx} = \frac{dy}{dx} \frac{dz}{dy}$. Now the shapes line up and our derivative has shape $m * p$ which is the shape that we expect, since the denominator (what we are differentiating with respect to) has shape $m$ and $z$ has shape $p$. 



#### Vector-valued backprop examples
- For $f = \sigma(w^Tx), w \in R^n, x \in R^n$, compute $\frac{df}{dw}$. 
 ![back4](back4.jpg)
 ![back5](back5.jpg)
- For $h = ReLU(WX + b)$ where $X$ is a batch of activations $ n * m$ and $W$ is a matrix denoting this hidden layer of size $m * k$ (so this layer has $k$ hidden neurons), we want to find $\frac{dL}{dW}$. Assume that we have the incoming gradient into this layer as $\frac{dL}{dh}$ which culd be the graident of the loss with repsct to this layer if this was the last layer or the incoming gradient from the next layer in the network.
- It turns out that $\frac{dL}{dW} = \mathbb{1}(Wx + b > 0)x^T$ and $\frac{dL}{db} = \mathbb{1}(Wx + b > 0)$. This has a nice interpretation: the gradient of our weights with respect to the loss is computed by letting each element of the incoming gradient pass through if the activations of this layer $relu(a)$ were active (greater than 0), otherwise not letting the gradient flow through, and then multiplying that result with our inputs into this layer $x$. 
![back6](back6.jpg)

### Weight Initialization
- Initializing the weights to all be $0$ will be bad, because this is basically telling the network to make predictions without actually being influenced by the inpust $x$. 
- What about small random weight initialization? 
- Works reasonably well for not very deep neural networks, but fails for very deep networks
- This is because the distribution of each layer's activations have less and less variance, but still a mean of zero, so they basically get clustered around $0$: 

- **Now think about what happens during the backwards pass**
- For each hidden layer, we showed that the gradient is $\frac{dl}{dW} = \mathbb{1}(a > 0)\frac{dL}{dh_{i+1}}h_i^T$. i.e. the gradient of the weights is the incoming gradient, multiplied by the relu (so zeroed where the relu was not active), **scaled by the inputs into this layer**. 
- And we just saw that as we get deeper and deeper, the inputs into the layer center around $0$, so the gradient will essentially be zero during learning: $$\frac{dL}{dW_i} = \mathbb{1} (W_i^T h_{i} > 0 ) \frac{dL}{dh_{i + 1}}h_i$$. Basically the gradient to the weights at the $i$th layer is dependent on the inputs $h_i$ into that layer, and since $h_i$ will be close to $0$ the gradient will be basically 0. 

#### What about Large Weight init?
- Also not great
- Causes the activations to explode
- Which results in an exploding gradient, because again the gradient of the weights at each layer is scaled by the activation -> what happens when you keep multiplying numbers greater than 1 together -> they blow up towards infinity. 
- Also since we're using relu, the gradient isn't constrained/scaled down. 

#### Xavier Init
- Paper that argues that the variance of activations across layers should be roughly teh same, and the same should hold for the backwards pass. 
- Means that we want $var(h_i) = var(h_j)$ for any hidden layers $i,j$ and $var(\nabla_{h_i} J) = var(\nabla_{h_j} J)$.

#### Xavier initialization derivation
- Requires several assumptions - linear units & independence. Full derivation is [here](https://github.com/rohan-varma/machine-learning-courses/blob/master/cs231n/notes/Explanation%20of%20Xavier%20Initialization.ipynb). 
- Basically, we get that to keep the variance the same across the forward pass, each layer's weights should be initialized with $\frac{1}{n_{in}}$, and if we want the same with the backwards pass, we should init our weights with $\frac{1}{n_{out}}$. Usually these are combined together, and the paper recommends an init of $\frac{2}{n_{in} + n_{out}}$. 

#### He initialization
- Motivation is that Xavier initialization generally doesn't work very well when you have relu units with very large deep networks (i.e. 20+ layers). Works well for the tanh neuron though. 
- **He initialization**: use the normalizer $\frac{2}{n_in}$ when using relu units. The motivation is that if the linear units that are input into the relu are equally likely to be negative or positive, then the output of the relu reduces the variance of the linear units by $\frac{1}{2}$, since relu would kill (about) half the units. So just add the extra factor of $2$ in the init to keep the variance the same. 
- Glorot and Bengio ultimately suggest that the weights should be drawn from a uniform distribution with parameters $ a= \frac{-\sqrt(6)}{n_{in}, n_{out}}, b = \frac{\sqrt(6)}{n_{in}, n_{out}}$.